In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from importlib import reload
ROOT_DIR = Path('..')

import AlexTurner.engineer_features.SOFA as SOFA
reload(SOFA)

SOFA = SOFA.SOFA(ROOT_DIR)

ModuleNotFoundError: No module named 'AlexTurner'

In [21]:
def get_sofa_score(df):
        """
        Refer to https://www.mdcalc.com/sequential-organ-failure-assessment-sofa-score#evidence
        
        Parameters:
        - df: A DataFrame containing: 
            Platelet_count [51265]
            Bilirubin_total [50885] 
            Creatinine [50912]
            Mean Arterial Pressure     # Requires Chart Events
        
        
        Returns:
        - SOFA score.
        """
        sofa = pd.DataFrame(index=df.index)
        sofa['Platelets'] = 0
        
        
        sofa['Platelets'][(400 <= df['51265']) & (df['51265'] < np.inf)] = 0
        sofa['Platelets'][(300 <= df['51265']) & (df['51265'] < 400)] = 1
        sofa['Platelets'][(200 <= df['51265']) & (df['51265'] < 300)] = 2
        sofa['Platelets'][(100 <= df['51265']) & (df['51265'] < 200)] = 3
        sofa['Platelets'][df['51265'] < 100] = 4
        sofa['Platelets'][df['51265'] == -999] = -999     # Set the value to 0 for rows == -999

        # sofa['Bilirubin'] = 0
        # sofa['Bilirubin'][df['50885'] < 1.2] = 0
        # sofa['Bilirubin'][(1.2 <= df['50885']) & ((df['50885']) <= 1.9)] = 1
        # sofa['Bilirubin'][(1.9 < df['50885']) & ((df['50885']) <= 5.9)] = 2
        # sofa['Bilirubin'][(5.9 < df['50885']) & ((df['50885']) <= 11.9)] = 3
        # sofa['Bilirubin'][(11.9 < df['50885'])] = 4
        # sofa['Bilirubin'][df['50885'] == -999] = 0     # Set the value to 0 for rows == -999

        sofa['Creatinine'] = 0
        sofa['Creatinine'][df['50912'] < 1.2] = 0
        sofa['Creatinine'][(1.2 <= df['50912']) & (df['50912'] < 1.9)] = 1
        sofa['Creatinine'][(1.9 <= df['50912']) & (df['50912'] < 3.5)] = 2
        sofa['Creatinine'][(3.5 <= df['50912']) & (df['50912'] < 5)] = 3
        sofa['Creatinine'][(df['50912'] >= 5)] = 4
        sofa['Creatinine'][df['50912'] == -999] = -999     # Set the value to 0 for rows == -999

        # sofa['MAP'][df['MAP'] < 70] = 1
        
        sofa_score = sofa.sum(axis=1)
        
        # Check if the sum is less than 0, if so set it to -999
        sofa_score[sofa_score < 0] = -999
        
        return sofa_score


In [17]:
df_input = pd.read_csv(Path(ROOT_DIR / 'data' / 'model_input' / 't0_v3_itemid_headers.csv'))
df_input

,SUBJECT_ID,HADM_ID,IS_SEPSIS,50882,50893,50912,50931,51221,51222,51237,...,51279,51006,51301,50818,50820,50821,51493,AGE,GENDER_NUM,SOFA
0,3,145834,0,25.0,8.2,3.2,91.0,30.2,10.0,1.3,...,3.44,53.0,12.7,-999.0,-999.0,-999.0,-999.0,76,0,4
1,4,185777,0,24.0,8.9,0.5,140.0,34.2,11.5,-999.0,...,3.80,9.0,9.7,-999.0,-999.0,-999.0,-999.0,47,1,2
2,6,107064,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,65,1,4
3,9,150750,0,30.0,-999.0,1.2,129.0,45.2,15.4,1.1,...,5.46,16.0,7.5,-999.0,-999.0,-999.0,-999.0,41,0,3
4,11,194540,0,25.0,-999.0,0.7,121.0,36.9,12.5,1.1,...,4.31,12.0,8.5,-999.0,-999.0,-999.0,-999.0,50,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47880,99985,176670,1,-999.0,-999.0,-999.0,-999.0,36.4,12.4,-999.0,...,4.20,-999.0,13.0,-999.0,-999.0,-999.0,-999.0,53,0,1
47881,99991,151118,1,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,47,0,4
47882,99992,197084,0,22.0,7.1,1.2,206.0,12.9,4.7,1.1,...,1.35,56.0,21.4,-999.0,-999.0,-999.0,-999.0,65,1,4
47883,99995,137810,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,88,1,4


In [22]:
df_input['SOFA'] = get_sofa_score(df_input)
df_input

,SUBJECT_ID,HADM_ID,IS_SEPSIS,50882,50893,50912,50931,51221,51222,51237,...,51279,51006,51301,50818,50820,50821,51493,AGE,GENDER_NUM,SOFA
0,3,145834,0,25.0,8.2,3.2,91.0,30.2,10.0,1.3,...,3.44,53.0,12.7,-999.0,-999.0,-999.0,-999.0,76,0,4
1,4,185777,0,24.0,8.9,0.5,140.0,34.2,11.5,-999.0,...,3.80,9.0,9.7,-999.0,-999.0,-999.0,-999.0,47,1,2
2,6,107064,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,65,1,-999
3,9,150750,0,30.0,-999.0,1.2,129.0,45.2,15.4,1.1,...,5.46,16.0,7.5,-999.0,-999.0,-999.0,-999.0,41,0,3
4,11,194540,0,25.0,-999.0,0.7,121.0,36.9,12.5,1.1,...,4.31,12.0,8.5,-999.0,-999.0,-999.0,-999.0,50,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47880,99985,176670,1,-999.0,-999.0,-999.0,-999.0,36.4,12.4,-999.0,...,4.20,-999.0,13.0,-999.0,-999.0,-999.0,-999.0,53,0,-999
47881,99991,151118,1,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,47,0,-999
47882,99992,197084,0,22.0,7.1,1.2,206.0,12.9,4.7,1.1,...,1.35,56.0,21.4,-999.0,-999.0,-999.0,-999.0,65,1,4
47883,99995,137810,0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,88,1,-999


In [ ]:
# Charlson Comorbidity Index

# Assume df_diagnoses is the DataFrame containing diagnoses information from MIMIC-III
# and charlson_mapping is a dictionary mapping ICD-9 codes to Charlson weights

def calculate_cci(df_diagnoses, charlson_mapping):
    patient_cci = {}
    
    for index, row in df_diagnoses.iterrows():
        subject_id = row['SUBJECT_ID']
        icd9_code = row['ICD9_CODE']
        
        weight = charlson_mapping.get(icd9_code, 0)
        
        if subject_id not in patient_cci:
            patient_cci[subject_id] = 0
            
        patient_cci[subject_id] += weight
        
    return patient_cci